## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## Compute the camera calibration using chessboard images

In [21]:
#Function to compute the camera calibration matrix and distortion coefficients for given a set of chessboard images.
def calibrateCamera(nx, ny):
    
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((nx*ny,3), np.float32)
    objp[:,:2] = np.mgrid[0:nx,0:ny].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Make a list of calibration images
    images = glob.glob('./camera_cal/calibration*.jpg')

    # Step through the list and search for chessboard corners
    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (nx,ny),None)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)
    
    #Camera calibration, given object points, image points, and the shape of the grayscale image:
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    
    return (mtx, dist)

## Used color transforms, gradients, etc., to create a thresholded binary image. 

In [22]:
#Function to create a thresholded binary image
def binaryThreshold(img):
  
    # Convert to HLS color space and separate the S channel
    # Note: img is the undistorted image
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    
    # Sobel x
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))

    # Threshold x gradient
    thresh_min = 20
    thresh_max = 100
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1

    # Threshold color channel
    s_thresh_min = 170
    s_thresh_max = 255
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh_min) & (s_channel <= s_thresh_max)] = 1

    # Stack each channel to view their individual contributions in green and blue respectively
    # This returns a stack of the two binary images, whose components you can see as different colors
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary))

    # Combine the two binary thresholds
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1
    
    return combined_binary

## Applying perspective transform to rectify binary image ("birds-eye view").

In [23]:
def perspectiveTransform(img, inverse):
    # Grab the image shape
    img_size = (img.shape[1], img.shape[0])
    # For source abd destination points I'm grabbing the outer four detected corners
    src = np.float32([[340,645],[575,465],[705,460],[975,635]])
    dst = np.float32([[340,645],[340,345],[975,340],[975,635]])
    
    #Compute the perspective transform, M, given source and destination points:
    if (inverse == False) :
        M = cv2.getPerspectiveTransform(src, dst)
    else:
        M = cv2.getPerspectiveTransform(dst, src)
    #Warp an image using the perspective transform, M:
    warpedimage = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    
    return warpedimage

## Applying Region of Interest

In [24]:
def region_of_interest(img, vertices):
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

## Detect lane pixels and fit to find the lane boundary

In [25]:
def extractLeftAndRightPixels(binary_warped):
    global vehicle_deviation
    global lines_detected
    global previous_left_fit
    global previous_right_fit
    
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]/2:,:], axis=0)
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
        
    previous_left_fit = left_fit
    previous_right_fit = right_fit
    lines_detected = True

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    left_line_x = left_fit[0] * binary_warped.shape[0] ** 2 + left_fit[1] * binary_warped.shape[0] + left_fit[2]
    right_line_x = right_fit[0] * binary_warped.shape[0] ** 2 + right_fit[1] * binary_warped.shape[0] + right_fit[2]
    vehicle_deviation = (((left_line_x + right_line_x) / 2.0) - (binary_warped.shape[1] / 2.0)) * xm_per_pix
    
    return (left_fitx, right_fitx)


## Skip the sliding window step if lines are known

In [26]:
def optimizedExtractLines(binary_warped):
    global vehicle_deviation
    
    global lines_detected
    global previous_left_fit
    global previous_right_fit

    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin = 100
    left_lane_inds = ((nonzerox > (previous_left_fit[0]*(nonzeroy**2) + previous_left_fit[1]*nonzeroy + previous_left_fit[2] - margin)) & (nonzerox < (previous_left_fit[0]*(nonzeroy**2) + previous_left_fit[1]*nonzeroy + previous_left_fit[2] + margin))) 
    right_lane_inds = ((nonzerox > (previous_right_fit[0]*(nonzeroy**2) + previous_right_fit[1]*nonzeroy + previous_right_fit[2] - margin)) & (nonzerox < (previous_right_fit[0]*(nonzeroy**2) + previous_right_fit[1]*nonzeroy + previous_right_fit[2] + margin)))  

# Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
# Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
   
    previous_left_fit = left_fit
    previous_right_fit = right_fit
    
# Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    left_line_x = left_fit[0] * binary_warped.shape[0] ** 2 + left_fit[1] * binary_warped.shape[0] + left_fit[2]
    right_line_x = right_fit[0] * binary_warped.shape[0] ** 2 + right_fit[1] * binary_warped.shape[0] + right_fit[2]
    vehicle_deviation = (((left_line_x + right_line_x) / 2.0) - (binary_warped.shape[1] / 2.0)) * xm_per_pix
    

    return (left_fitx, right_fitx)


## Determining the curvature of the lane

In [27]:
def findCurvature(ploty, left_fitx, right_fitx) :
    global ym_per_pix 
    global xm_per_pix 

    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(ploty*ym_per_pix, left_fitx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, right_fitx*xm_per_pix, 2)
    # Calculate the new radii of curvature
    y_eval = np.max(ploty)
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
 
    return(left_curverad, right_curverad)

## Running the pipeline on test images. Please check the output in "output_images" folder

In [28]:
def test_images():
    global mtx
    global dist
    
    #Step 2: Applying a distortion correction to test images
    # Make a list of test images
    images = glob.glob('./test_images/test*.jpg')

    # Step through the list and search for chessboard corners
    for fname in images:
        img = cv2.imread(fname)
        input_image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        basename = os.path.basename(fname)

        #Undistorting a test image:
        undstimage = cv2.undistort(img, mtx, dist, None, mtx)

    #Step 3: Used color transforms, gradients, etc., to create a thresholded binary image.    
        binimage = binaryThreshold(undstimage)

        #Applying Region of interest
        imgWidth = binimage.shape[1]
        imgHeight = binimage.shape[0]
        vertices =  np.array([[(0, imgHeight),  # bottom left
                               (imgWidth * 0.47, imgHeight * 0.60), # top left
                               (imgWidth * 0.53, imgHeight * 0.60), # top right
                               (imgWidth, imgHeight)]], dtype=np.int32) # bottom right

        roioutput = region_of_interest(binimage, vertices)

    #Step 4: Apply a perspective transform to rectify binary image ("birds-eye view").
        warpedimage = perspectiveTransform(roioutput, False)
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
        ax1.set_title('Stacked thresholds')
        ax1.imshow(undstimage)
        ax2.set_title('Combined S channel and gradient thresholds')
        ax2.imshow(warpedimage, cmap='gray')
        ofname = "./output_images/" + os.path.splitext(basename)[0] + "_threshold_warped.png"
        plt.savefig(ofname)

    #Step 5: Detect lane pixels and fit to find the lane boundary  
        left_fitx, right_fitx = extractLeftAndRightPixels(warpedimage)

    #Step 6: Determine the curvature of the lane and vehicle position with respect to center
        ploty = np.linspace(0, warpedimage.shape[0]-1, warpedimage.shape[0] )
        left_rad, right_rad = findCurvature(ploty, left_fitx, right_fitx)


    #Step 7: Warp the detected lane boundaries back onto the original image.
        # Create an output image to draw on and  visualize the result
        out_img = np.dstack((warpedimage, warpedimage, warpedimage))*255
        left_line = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
        right_line = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
        binary_warped_zero = np.zeros_like(warpedimage).astype(np.uint8)

        lanes_warped = np.dstack((binary_warped_zero, binary_warped_zero, binary_warped_zero))
        lane_pts = np.hstack((left_line, right_line))

        # Draw the lane onto the warped blank image
        cv2.fillPoly(lanes_warped, np.int_([lane_pts]), (0,255, 0))
        warped_result = cv2.addWeighted(out_img, 1, lanes_warped, 0.3, 0)
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
        ax1.set_title('Stacked thresholds')
        ax1.imshow(undstimage)
        ax2.set_title('Detected Lane Boundaries')
        ax2.imshow(warped_result)
        ofname = "./output_images/" + os.path.splitext(basename)[0] + "_lane_boundaries.png"
        plt.savefig(ofname)

    #Step 8: Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position
        unwarped_result = cv2.addWeighted(undstimage, 1, perspectiveTransform(lanes_warped, True), 0.3, 0)
        output_img = cv2.cvtColor(unwarped_result, cv2.COLOR_BGR2RGB)

        cv2.putText(output_img, 'Vehicle Deviation: {:3.3f}m'.format(vehicle_deviation), (50, 50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (200,255,155), 2, cv2.LINE_AA)
        cv2.putText(output_img, 'Curvature: Left {:3.3f}m Right {:3.3f}m'.format(left_rad, right_rad), (50, 100), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (200,255,155), 2, cv2.LINE_AA)
        ax1.set_title('Input Image')
        ax1.imshow(input_image)
        ax2.set_title('Output Image')
        ax2.imshow(output_img)
        ofname = "./output_images/" + os.path.splitext(basename)[0] + "_final_output.png"
        plt.savefig(ofname)

    print("\nSuccess : Completed the processing of all the input images, check 'output_images' folder \n")

## Process image to process each frame of the input video

In [29]:
def process_image(img):
        global mtx
        global dist
        
        global lines_detected
        global previous_left_fit
        global previous_right_fit
        global previous_left_fitx
        global previous_right_fitx
        
        undstimage = cv2.undistort(img, mtx, dist, None, mtx) 
        binimage = binaryThreshold(undstimage)

        #Applying Region of interest
        imgWidth = binimage.shape[1]
        imgHeight = binimage.shape[0]
        vertices =  np.array([[(0, imgHeight),  # bottom left
                               (imgWidth * 0.47, imgHeight * 0.60), # top left
                               (imgWidth * 0.53, imgHeight * 0.60), # top right
                               (imgWidth, imgHeight)]], dtype=np.int32) # bottom right

        roioutput = region_of_interest(binimage, vertices)

        #Step 4: Apply a perspective transform to rectify binary image ("birds-eye view").
        warpedimage = perspectiveTransform(roioutput, False)
        
        
        #Step 5: Detect lane pixels and fit to find the lane boundary  
        if lines_detected is False :
            left_fitx, right_fitx = extractLeftAndRightPixels(warpedimage)
            previous_left_fitx = left_fitx
            previous_right_fitx = right_fitx
        else:
            left_fitx, right_fitx = optimizedExtractLines(warpedimage)
            diff_left_fitx = abs(previous_left_fitx - left_fitx)
            diff_right_fitx = abs(previous_right_fitx - right_fitx)
            avg_diff_left = np.average(diff_left_fitx, axis=0)
            avg_diff_right = np.average(diff_right_fitx, axis=0)
            if (avg_diff_left > 30):
                left_fitx = previous_left_fitx
                lines_detected = False
            if (avg_diff_right > 30):
                right_fitx = previous_right_fitx
                lines_detected = False
                    
        #Step 6: Determine the curvature of the lane and vehicle position with respect to center
        ploty = np.linspace(0, warpedimage.shape[0]-1, warpedimage.shape[0] )
        left_rad, right_rad = findCurvature(ploty, left_fitx, right_fitx)
        if (abs(left_rad-right_rad) > 100) : 
            lines_detected = False
    
    #Step 7: Warp the detected lane boundaries back onto the original image.
        # Create an output image to draw on and  visualize the result
        out_img = np.dstack((warpedimage, warpedimage, warpedimage))*255
        left_line = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
        right_line = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
        binary_warped_zero = np.zeros_like(warpedimage).astype(np.uint8)

        lanes_warped = np.dstack((binary_warped_zero, binary_warped_zero, binary_warped_zero))
        lane_pts = np.hstack((left_line, right_line))

        # Draw the lane onto the warped blank image
        cv2.fillPoly(lanes_warped, np.int_([lane_pts]), (0,255, 0))
        warped_result = cv2.addWeighted(out_img, 1, lanes_warped, 0.3, 0)
        
        unwarped_result = cv2.addWeighted(undstimage, 1, perspectiveTransform(lanes_warped, True), 0.3, 0)
      
        cv2.putText(unwarped_result, 'Vehicle Deviation : {:3.3f}m'.format(vehicle_deviation), (50, 50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (200,255,155), 2, cv2.LINE_AA)
        cv2.putText(unwarped_result, 'Curvature : Left {:3.3f}m Right {:3.3f}m'.format(left_rad,right_rad), (50, 100), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (200,255,155), 2, cv2.LINE_AA)
        return unwarped_result
    

## Main for testing the input images and video
## Please check ouput_project_video.mp4 and "output_images" folder

In [31]:
import numpy as np
import cv2
import glob
import os
import matplotlib.pyplot as plt
from moviepy.editor import VideoFileClip
#%matplotlib qt

vehicle_deviation = 0.0
lines_detected = False
previous_left_fit = None
previous_right_fit = None
previous_left_fitx = None
previous_right_fitx = None

#Define conversions in x and y from pixels space to meters
ym_per_pix = 30/720 # meters per pixel in y dimension
xm_per_pix = 3.7/700 # meters per pixel in x dimension

#Step 1: To compute the camera calibration matrix and distortion coefficients for given a set of chessboard images.
mtx, dist = calibrateCamera(9, 6)

#Testing calibration for one image
img = cv2.imread("./camera_cal/calibration3.jpg")
undstimage = cv2.undistort(img, mtx, dist, None, mtx)
f, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(img)
ax1.set_title('Distorted')
ax2.imshow(undstimage)
ax2.set_title('Undistorted ')
ofname = "./output_images/" + "camera_calibration_output.png"
plt.savefig(ofname)

#Testing input images
test_images()

#Testing input video
lines_detected = False
previous_left_fit = None
previous_right_fit = None
previous_left_fitx = None
previous_right_fitx = None
project_output = 'output_project_video.mp4'
clip1 = VideoFileClip("project_video.mp4")
out_clip = clip1.fl_image(process_image) 
out_clip.write_videofile(project_output, audio=False)
print("\nSuccess : Completed the processing of the input video, check output video 'output_project_video.mp' \n")


Success : Completed the processing of all the input images, check 'output_images' folder 

[MoviePy] >>>> Building video output_project_video.mp4
[MoviePy] Writing video output_project_video.mp4


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [04:09<00:00,  4.83it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_project_video.mp4 


Success : Completed the processing of the input video, check output video 'output_project_video.mp' 

